### Clustering the TFs
The code below is to use the peak detector to find the peaks and valleys. Afterward cluster the TFs based on the number of peaks and valleys detected. Output will be folders of TFs based on 'peaks_valleys' and the associated TF barcodes. Also included in output folder is the metaplot histogram and heatmap of all TFs that falled into each cluster.

In [1]:
import csv

def read_mds_file(file_path):
    """Read a csv file containg MDS data into two dimensional array (list of lists)"""
    ids = []
    data = []
    with open(file_path) as csvfile:
        readCSV = csv.reader(csvfile, delimiter=',')
        i = 0 
        for row in readCSV:
            if i > 0: # skip the header row
                data.append(list(map(float, row[1:])))
                ids.append(row[0])
            i += 1
    return ids, data

def read_sim_file(file_path):
    """Read a txt file containg MDS data into two dimensional array (list of lists)"""
    ids = []
    data = []
    with open (file_path, 'r') as txtfile:
        readTxt = csv.reader(txtfile, delimiter=';')
        i = 0 
        for row in readTxt:
            if i > 0: # skip the header row
                data.append(list(map(float, row[1:])))
                ids.append(row[0])
            i += 1
    return ids, data


In [ ]:
import os
import numpy as np
import pandas as pd
import pylab
import matplotlib.pyplot as plt 
import seaborn as sb
import scipy.signal
from scipy.signal import find_peaks
from scipy.signal import savgol_filter
from scipy.signal import lfilter
from statistics import *
from itertools import groupby


def _smooth_data(raw_data_row):
    """smooth the data to prepare for usage in a peak finding algorythm"""
    # data_row_tmp0 = data_row
    def chunks1(lst, n):
        """Yield successive n-sized chunks from lst."""
        for i in range(0, len(lst), n):
            yield lst[i:i + n]
    def chunks2(lst, n):
        """rolling cluster buckets"""
        for i in range(0, len(lst) - n, 1):
            yield lst[i:i + n]
    
    bucketed_data_row = []
    smoothed_data_row = []
    
    #break data into buckets of 5 units each
    #[[0,5], [6, 10], [11,15], ...]
    #get the sum of each bucket
    #generate new dataset with bucketed data
    #this allows us to eccentuate peak data and create larger gaps between signal and noise
    for i in chunks1(raw_data_row, 10):
        bucketed_data_row.append(sum(i))
        
    #generate rolling buckets
    #[[0,n], [1,n+1], [2, n+2], ...]
    #get the mean of each subbucket
    #this smooths out noisy peaks and imporoves detection rates
    #this lets us filter out 1 unit wide 'peaks' in the middle of noise
    #this also makes real peaks easier to detect by smoothing out 'twin tower' peaks
    for i in chunks2(bucketed_data_row, 10):
        smoothed_data_row.append(mean(i))
        
    return smoothed_data_row


def _plot_raw_data(raw_data_row, image_path):
    plt.rcParams["figure.figsize"] = (10,3)
    plt.plot(np.array(raw_data_row))
    plt.savefig(image_path)
    plt.close()

def plot_heatmap(data, image_path):
    #plt.rcParams["figure.figsize"] = (10,3)
    heat_map = sb.heatmap([data,], cmap="YlGnBu")
    plt.savefig(image_path)
    plt.close()

# def _plot_peak_data():
#     meanArr = np.array([my] * len(y))
#     medianArr = np.array([mdy] * len(y))
#     plt.rcParams["figure.figsize"] = (10,3)
#     plt.plot(y);
#     plt.plot(meanArr, color="red")
#     plt.plot(medianArr, color="red", linestyle='dashed')
#     plt.plot(meanArr + sy15, color="orange")
#     plt.plot(meanArr - sy15, color="orange")

#========================

def find_specified_extrema(rowNum, smoothed_data, peakScan, troughScan):
    # plt.rcParams["figure.figsize"] = (10,3)
    # plt.plot(np.array(smoothed_data[rowNum]))
    # plt.show()
    
    peaks = []
    troughs = []
    ymax = max(smoothed_data)
    my = mean(smoothed_data)
    mdy = median(smoothed_data)
    sy = stdev(smoothed_data)
    sy15 = sy * 1.5
    y = np.array(smoothed_data)
    
    #invert data before finding peaks
    peak_properties = None
    trough_properties = None
    if peakScan == True:
        #returns indexes of y that are detected as peaks
        peaks, peak_properties = find_peaks(y, height=my+sy15, distance = 20, prominence=4, width=3)
        # plt.plot(peaks, y[peaks], "xm");

    if troughScan == True:
        yinv = ymax - y
        myInv = ymax - my
        troughs, trough_properties = find_peaks(yinv, height=myInv+sy15, distance = 20, prominence=4, width=3)
        # plt.plot(troughs, y[troughs], "xm");

    # plt.show()
    
    return len(peaks), len(troughs), peak_properties, trough_properties

    
#dynamically switch between peak/trough detection
#uses stddev outlier detection to determine if there is a substantial negative peak
#if n points are less than mean(y) - stddev(y), then the histo will be treated asna trough
def find_extrema(raw_data_row):
    smoothed_data = _smooth_data(raw_data_row)
    my = mean(smoothed_data)
    sy = stdev(smoothed_data)
    sy15 = sy * 1.5
    
    #generate signal array
    #square signal based on outlier status
    peakScan = False
    troughScan = False
    scanThresh = 5
    signals = []
    for i in smoothed_data:
        if i > (my + sy15):
            signals.append(1)
        elif i < (my - sy15):
            signals.append(-1)
        else:
            signals.append(0)
    #group by signal level and get length of continuous signal
    #[0, 0, 1, 1, 1, 0, 1, 1] => [(0,2),(1,3),(0,1),(1,2)]
    #if there is a long continous signal, then that indicates peak/trough
    grouped_L = [(k, sum(1 for i in g)) for k,g in groupby(signals)]
    for (sigVal, count) in grouped_L:
        if sigVal == 1 and count > scanThresh:
            peakScan = True
        if sigVal == -1 and count > scanThresh:
            troughScan = True
    
    return find_specified_extrema(rowNum, smoothed_data, peakScan, troughScan)


rowNum = 0

#from read_mds_file import read_mds_file
# ids, raw_data = read_mds_file('/scratch/Shares/dowell/md_score_paper/mdscore_tsv_files/human/recent/SRR1554311_MDS.csv')
ids, raw_data = read_sim_file('levandow_simulated_md_scores_5000.txt')
peak_bins_file = open('peak_bins.csv', 'w')
peak_bins_file.write('peak,trough,tf\n')
    
rows_by_counts = {}
for i in range(0, len(raw_data)):
    raw_data_row = raw_data[i]
    # _plot_raw_data(raw_data_row)
    pCount, tCount, peak_properties, trough_properties = find_extrema(raw_data_row)
    if (pCount, tCount) not in rows_by_counts:
        rows_by_counts[(pCount, tCount)] = []
    rows_by_counts[(pCount, tCount)].append(i)
    id = ids[i].replace('HO_','').replace('_HUMAN.H10MO','')
    peak_bins_file.write('{},{},{}\n'.format(pCount, tCount, id))
    
peak_bins_file.close()
# print(rows_by_counts)

for counts, rownums in rows_by_counts.items():
    peak_count, trough_count = counts
    print(peak_count, trough_count, rownums)
    output_folder = 'output/{}_{}/'.format(peak_count, trough_count)
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)  
        
    
    # var to store aggregated for heat map
    # should the size of the raw data intialized to zeros
    agg_norm_data = [0 for _ in range(len(raw_data[0]))]
    for rownum in rownums:
        output_filename = '{}{}.png'.format(output_folder,ids[rownum])
        output_filename = output_filename.replace('HO_','')
        output_filename = output_filename.replace('_HUMAN.H10MO','')
        _plot_raw_data(raw_data[rownum], output_filename)
        # normalize the raw data
        raw_total = sum(raw_data[rownum])
        if raw_total > 0:
            normalized_raw_data = [v/raw_total for v in raw_data[rownum]]
         # add the normalized data to aggregated data
        for i, value in enumerate(normalized_raw_data):
            agg_norm_data[i] += value
    # plot the aggregated normalized data and store in output_folder
    _plot_raw_data(agg_norm_data, '{}meta_histogram.png'.format(output_folder))

    # plot heat map of the aggregated normalized data and store in output_folder
    plot_heatmap(agg_norm_data, '{}meta_heatmap.png'.format(output_folder))
    

0 0 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 

### Clusters based on peaks and valleys
Output of peak_bins.csv 

In [4]:
import pandas as pd

df = pd.read_csv('peak_bins.csv')
df = df.sort_values(by=['peak', 'trough'])
df['bin'] = pd.Series(['' for x in range(len(df.index))])

for index in df.index:
    bin_value = df.peak.astype(str).str.cat(df.trough.astype(str), sep='_')
    df.bin = bin_value 

num_bin = df['bin'].nunique()
print("number of unique bins", num_bin)

tf_bin = df.groupby('bin').nunique()
print(tf_bin)

number of unique bins 253
         peak  trough  tf
bin                      
100_100     1       1   1
102_109     1       1   1
10_10       1       1   1
12_12       1       1   1
13_13       1       1   2
...       ...     ...  ..
73_74       1       1   1
76_77       1       1   1
78_79       1       1   1
94_94       1       1   1
9_9         1       1   1

[253 rows x 3 columns]
